In [5]:
PATH = "/home/mdobrzanski/playground/IET/NLP/ustawy/"
%config Completer.use_jedi = False

In [192]:
import regex as re
import os
from morfeusz2 import Morfeusz 
import matplotlib.pyplot as plt
import elasticsearch
import elasticsearch_dsl
import os
import re
import spacy
import time
from collections import defaultdict, Counter, OrderedDict
from functools import reduce
import math
import pandas as pd

In [7]:
morf = Morfeusz()

In [8]:
laws = {}

for filename in os.listdir(PATH):
    with open(PATH+filename, 'r') as file:
        tmp = file.read()
    laws[filename] = tmp

# 1

In [9]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [10]:
laws = {k: remove_html_tags(v) for k, v in laws.items()}

In [11]:
nlp = spacy.load('pl_core_news_sm')
tokenizer = nlp.tokenizer

In [12]:
law_tokens = {}

for k, v in laws.items():
    law_tokens[k] = [token.text.lower() for token in tokenizer(v)]

# 2

In [124]:
def count_n_grams_text(text, n=2):
    result = defaultdict(int)
    text = list(map(lambda x: x.lower(), text))
    for i in range(n,len(text)):
        result[' '.join(text[i-n:i])] += 1
    return result

def count_n_grams_dict(dictionary, n=2):
    result = list(map(lambda x: Counter(count_n_grams_text(x, n)), dictionary.values()))
    result = sum(result, Counter())
    return (defaultdict(int, result))
#     result = defaultdict(int)
#     for k, v in dictionary.items():
#         result = defaultdict(int, Counter(result)+Counter(count_n_grams_text(v, n)))
#     return result

In [125]:
bigrams = count_n_grams_dict(law_tokens, 2)

# 3

In [127]:
def clean_n_grams(n_grams, n):
    to_del = []
    for k in n_grams.keys():
        if not k.replace(' ', '').isalpha():
            to_del.append(k)
        elif len(k.split(' ')) != n:
            to_del.append(k)
    for k in to_del:
        del n_grams[k]

In [128]:
clean_n_grams(bigrams, 2)

# 4

In [130]:
unigrams = count_n_grams_dict(law_tokens, 1)

In [131]:
clean_n_grams(unigrams, 1)

In [132]:
unigram_count = sum(unigrams.values())    
bigram_count = sum(bigrams.values()) 

In [137]:
def pmi(n_gram, unigram_count, n_gram_count, unigrams, n_grams):
    tokens = n_gram.split(' ')
    tokens_p = [unigrams[token]/unigram_count for token in tokens]
    n_gram_p = n_grams[n_gram] / n_gram_count
    return math.log2(n_gram_p / reduce(lambda x, y: x*y , tokens_p, 1))

In [138]:
pmi('z dnia', unigram_count, bigram_count, unigrams, bigrams)

4.97607494079538

In [139]:
bigrams_pmi = {k: pmi(k, unigram_count, bigram_count, unigrams, bigrams) for k in bigrams.keys()}

# 5

In [141]:
bigrams_pmi = OrderedDict(bigrams_pmi)

In [146]:
bigrams_pmi = OrderedDict(sorted(bigrams_pmi.items(), key=lambda x: x[1], reverse=True))

In [150]:
list(bigrams_pmi.items())[:10]

[('kwotom wydatkowanym', 22.305257090081515),
 ('jan kanty', 22.305257090081515),
 ('marka karpia', 22.305257090081515),
 ('aleksander małachowski', 22.305257090081515),
 ('wyrządziłoby niewspółmierną', 22.305257090081515),
 ('aleksandrów kujawski', 22.305257090081515),
 ('jezus nadchodzi', 22.305257090081515),
 ('bystrzyca kłodzka', 22.305257090081515),
 ('bielsk podlaski', 22.305257090081515),
 ('dubicze cerkiewne', 22.305257090081515)]

# 6

In [151]:
filterd_bigrams_pmi = OrderedDict({k: v for k, v in bigrams_pmi.items() if bigrams[k]>=5})

In [152]:
filterd_bigrams_pmi = OrderedDict(sorted(filterd_bigrams_pmi.items(), key=lambda x: x[1], reverse=True))

In [153]:
list(filterd_bigrams_pmi.items())[:10]

[('ręcznego miotacza', 19.983328995194153),
 ('świeckie przygotowujące', 19.983328995194153),
 ('młynki młotkowe', 19.983328995194153),
 ('młyny kulowe', 19.983328995194153),
 ('otworami wiertniczymi', 19.983328995194153),
 ('zaszkodzić wynikom', 19.983328995194153),
 ('środa wlkp', 19.983328995194153),
 ('klęskami żywiołowymi', 19.983328995194153),
 ('grzegorz schetyna', 19.983328995194153),
 ('obcowania płciowego', 19.983328995194153)]

# 7

In [156]:
def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])


def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

In [160]:
def count_token_pos(n_grams, n):
    n_grams_token_pos = {i: defaultdict(int) for i in range(n)}
    for ngram, count in n_grams.items():
        tokens = ngram.split(' ')
        for i, t in enumerate(tokens):
            n_grams_token_pos[i][t] += count
    return n_grams_token_pos

In [165]:
first_count, second_count = count_token_pos(bigrams, 2).values()

In [171]:
bigrams_llr = {}

for bigram, count in bigrams.items():
    first_token, second_token = bigram.split(' ')
    k11 = count
    k12 = second_count[second_token] - count
    k21 = first_count[first_token] - count
    k22 = bigram_count - (k11 + k12 + k21)
    bigrams_llr[bigram] = llr_2x2(k11, k12, k21, k22)

# 8

In [172]:
bigrams_llr = OrderedDict(sorted(bigrams_llr.items(), key=lambda x: x[1], reverse=True))

In [173]:
list(bigrams_llr.items())[:10]

[('mowa w', 169181.7376764058),
 ('w art', 113470.57289345935),
 ('otrzymuje brzmienie', 112413.77084682393),
 ('których mowa', 107451.54559534334),
 ('o których', 84683.54216866661),
 ('w ust', 82329.59548457037),
 ('którym mowa', 70218.26537712),
 ('dodaje się', 63860.42135516362),
 ('do spraw', 57706.31135198218),
 ('o którym', 55280.024054311216)]

# 9

In [174]:
trigrams = count_n_grams_dict(law_tokens, 3)
clean_n_grams(trigrams, 3)

# 10

In [175]:
trigram_count = sum(trigrams.values()) 

In [180]:
trigrams_pmi = {k: pmi(k, unigram_count, trigram_count, unigrams, trigrams) for k in trigrams.keys()}

In [181]:
filterd_trigrams_pmi = OrderedDict({k: v for k, v in trigrams_pmi.items() if trigrams[k]>=5})

In [182]:
filterd_trigrams_pmi = OrderedDict(sorted(filterd_trigrams_pmi.items(), key=lambda x: x[1], reverse=True))

In [183]:
list(filterd_trigrams_pmi.items())[:10]

[('profilem zaufanym epuap', 37.02371880914245),
 ('finałowego turnieju mistrzostw', 36.643238531899975),
 ('przedwczesnego wyrębu drzewostanu', 36.46968613495318),
 ('potwierdzonym profilem zaufanym', 36.3507001317561),
 ('piłce nożnej uefa', 36.292147949400984),
 ('cienką sierścią zwierzęcą', 35.94314032045734),
 ('szybkiemu postępowi technicznemu', 35.878297365072584),
 ('turnieju mistrzostw europy', 35.877703785537),
 ('grożącą jemu samemu', 35.63679612078843),
 ('wypalonym paliwem jądrowym', 35.562795539344656)]

In [184]:
first_count, second_count, third_count = count_token_pos(trigrams, 3).values()

In [185]:
trigrams_llr = {}

for trigram, count in trigrams.items():
    first_token, second_token, third_token = trigram.split(' ')
    k11 = count
    k1 = first_count[first_token] - count
    k2 = second_count[second_token] - count
    k3 = third_count[third_token] - count
    k33 = trigram_count - (k11 + k12 + k21)
    trigrams_llr[trigram] = llr_2x2(k11, k2, k1, k33)/3 + llr_2x2(k11, k3, k2, k33)/3 + llr_2x2(k11, k3, k1, k33)/3

In [186]:
trigrams_llr = OrderedDict(sorted(trigrams_llr.items(), key=lambda x: x[1], reverse=True))

In [187]:
list(trigrams_llr.items())[:10]

[('o których mowa', 74867.2374197212),
 ('których mowa w', 66024.52174543576),
 ('mowa w ust', 57527.66018026951),
 ('o którym mowa', 47180.656883170406),
 ('mowa w art', 44345.515028177586),
 ('którym mowa w', 41445.0919144674),
 ('minister właściwy do', 31989.829517168364),
 ('właściwy do spraw', 31614.3212571622),
 ('o której mowa', 25549.948442581925),
 ('w drodze rozporządzenia', 24395.458024731957)]

# 11

In [205]:
table2 = [[a, b, c, d] for (a, b),  (c, d) in zip(list(filterd_bigrams_pmi.items())[:20], list(bigrams_llr.items())[:20])]

In [206]:
pd.DataFrame(table2, columns=['PMI token', 'PMI score', 'LLR token', 'LLR score'])

,PMI token,PMI score,LLR token,LLR score
0,ręcznego miotacza,19.983329,mowa w,169181.737676
1,świeckie przygotowujące,19.983329,w art,113470.572893
2,młynki młotkowe,19.983329,otrzymuje brzmienie,112413.770847
3,młyny kulowe,19.983329,których mowa,107451.545595
4,otworami wiertniczymi,19.983329,o których,84683.542169
5,zaszkodzić wynikom,19.983329,w ust,82329.595485
6,środa wlkp,19.983329,którym mowa,70218.265377
7,klęskami żywiołowymi,19.983329,dodaje się,63860.421355
8,grzegorz schetyna,19.983329,do spraw,57706.311352
9,obcowania płciowego,19.983329,o którym,55280.024054


In [207]:
table3 = [[a, b, c, d] for (a, b),  (c, d) in zip(list(filterd_trigrams_pmi.items())[:20], list(trigrams_llr.items())[:20])]

In [208]:
pd.DataFrame(table3, columns=['PMI token', 'PMI score', 'LLR token', 'LLR score'])

,PMI token,PMI score,LLR token,LLR score
0,profilem zaufanym epuap,37.023719,o których mowa,74867.237420
1,finałowego turnieju mistrzostw,36.643239,których mowa w,66024.521745
2,przedwczesnego wyrębu drzewostanu,36.469686,mowa w ust,57527.660180
3,potwierdzonym profilem zaufanym,36.350700,o którym mowa,47180.656883
4,piłce nożnej uefa,36.292148,mowa w art,44345.515028
5,cienką sierścią zwierzęcą,35.943140,którym mowa w,41445.091914
6,szybkiemu postępowi technicznemu,35.878297,minister właściwy do,31989.829517
7,turnieju mistrzostw europy,35.877704,właściwy do spraw,31614.321257
8,grożącą jemu samemu,35.636796,o której mowa,25549.948443
9,wypalonym paliwem jądrowym,35.562796,w drodze rozporządzenia,24395.458025



# 12

Answer the following questions:

    Why do we have to filter the bigrams, rather than the token sequence?
    
    - W celu uniknięcia łączenia oddalonych od siebie słów w n-gramy
    
    
    Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
    
    - Ciężko powiedzieć co znaczy lepiej, moim zdaniem dla obydwu zagadnień bardziej znaczące wyniki daje LLR
    
    
    What types of expressions are discovered by the methods.
    
    - Przez PMI raczej n-gramy zawierające specyficzne, rzadko występujące słowe, przez LLR raczej bardzo częste n-gramy
    
    
    Can you devise a different type of filtering that would yield better results?
    
    - Myślę że mogłaby pomóc lematyzacja słów w n-gramach
